In [14]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message(
    "Hi I'm Shin Woo Lee, I live in South Korea", "Wow that is so cool!"
)

memory.load_memory_variables({"input": "Who is Shin Woo Lee?"})

{'history': [SystemMessage(content='On Shin Woo Lee: Shin Woo Lee lives in South Korea.')]}